<a href="https://colab.research.google.com/github/simonbustamante/genoma-pca-and-k-means/blob/master/Caso_de_estudio_1_2_An%C3%A1lisis_de_la_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de estudio 1.2: Análisis de la LDA (*Latent Dirichlet Allocation*)

Librerías a importar:

In [ ]:
import sys, re, time, string, random, csv, argparse
import requests
import numpy as n
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import psi

#Librerías de web scraping
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm

#Librerías de NLP
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import wordpunct_tokenize


print('\n¡Librerías importadas con éxito!')

# Generación de la base de datos (*Web Scraping*)

En primer lugar debemos obtener la lista de profesores del departamento de EECS, y el lab al que pertenecen. Para ello podemos usar librerías de _Web scraping_ como `BeautifulSoup`. Este tipo de librerías ofrecen funciones para explorar el código fuente de páginas web, y obtener información de su contenido.

In [ ]:
url_eecs_fac = 'https://www.eecs.mit.edu/role/faculty/'
h = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15"}
html_data = requests.get(url_eecs_fac, headers=h).text
soup = BeautifulSoup(html_data, 'html.parser')

#Guardando el nombre y el laboratorio en dos listas
fac = []
labs = []
entries = soup.find_all('div', class_='people-entry')
for entry in entries:
    name = entry.find('h5').find('a').text.strip()
    lab = entry.find('div', class_='people-research').find_all('a')
    lab_names = [a.text for a in lab]
    fac.append(name)
    labs.append(lab_names)
fac_dept = list(zip(fac, labs))
fac_dept = [(tup[0], ', '.join(tup[1])) for tup in fac_dept if tup[1]]


print('Número de profesores de EECS: {}'.format(len(fac_dept)))
print(fac_dept[:5],'...')

In [ ]:
html_data = requests.get(url_eecs_fac).text
html_data
soup = BeautifulSoup(html_data, 'html.parser')


Definimos una función para obtener todos los artículos de _arXiv_ de un autor determinado.

In [ ]:
def get_articles_for_author(author):
    base_url = 'https://arxiv.org/search/?query=%22{name}%22&searchtype=author&abstracts=show&order=-announced_date_first&size=200'
    author_query_url = base_url.format(name= author.replace(' ','+'))
    query_result = requests.get(author_query_url).text
    soup = BeautifulSoup(query_result, 'html.parser')
    articles = soup.find_all(class_ = 'arxiv-result')

    ids = [el.find(class_ = 'list-title is-inline-block').find('a').text.strip('arXiv:') for el in articles]
    ids = [el.split('/')[1] if el.find('/')>=0 else el for el in ids]

    titles = [el.find(class_ = 'title is-5 mathjax').text.strip(' \n') for el in articles]
    authors = [[author.text for author in el.find(class_ = 'authors').find_all('a')] for el in articles]
    abstracts = []
    for el in articles:
        try:
            abstracts.append(el.find(class_ = 'abstract-full has-text-grey-dark mathjax').text[9:-16])
        except:
            abstracts.append(el.find(class_ = 'abstract-short has-text-grey-dark mathjax').text[9:-16])
    urls = [el.find(class_ = 'list-title is-inline-block').find('a')['href'] for el in articles]

    return ids, titles, authors, urls, abstracts

author = fac_dept[23][0]
print('Ejemplo de un artículo de {}: \n'.format(author))
ids, titles, authors, urls, abstracts = get_articles_for_author(author)
i = 0
print('arXiv ID: {} (url: {} )'.format(ids[i],urls[i]))
print('Título: {}'.format(titles[i]))
print('Autores: {}'.format(authors[i]))
print('---\n{}\n---'.format(abstracts[i]))

Una vez tenemos dicha función definida y hemos comprobado que funciona, podemos iterar la lista de profesores para obtener los artículos de cada profesor.

In [ ]:
ids = []
titles = []
authors = []
labs = []
EECS_facs = []
urls = []
abstracts = []
print('Descargando artículos de cada profesor:')
t0 = time.time()
for i,fac in enumerate(fac_dept):
    id_list, title_list, author_list, url_list, abstract_list  = get_articles_for_author(fac[0])
    ids += id_list
    titles += title_list
    authors += author_list
    urls += url_list
    abstracts += abstract_list
    labs += [fac[1]]*len(id_list)
    EECS_facs += [fac[0]]*len(id_list)
    if round(i/10) == i/10:
        print('{}/{} autores'.format(i,len(fac_dept)))
tf = time.time()
print('{} artículos descargados en {:.2f}s'.format(len(ids),tf-t0))

Podemos guardar toda la información en una tabla de `pandas`, la siguiente celda muestra cómo hacerlo:

In [ ]:
df = pd.DataFrame({'id':ids,'title':titles,'EECS_prof':EECS_facs,'lab':labs,'authors':authors,'url':urls,'abstract':abstracts})
df[['id','lab']] = df[['id','lab']].drop_duplicates()
df.head()

## Obtención del vocabulario

Para poder utilizar las funciones proporcionadas en el guión del caso de estudio se necesita un archivo de vocabulario en formato `.csv`. Podemos obtenerlo del repositorio del estudio original de LDA SVI:

In [ ]:
url = 'https://raw.githubusercontent.com/blei-lab/onlineldavb/master/dictnostops.txt'
vocab_list = pd.Series(requests.get(url).text.split('\n')[:-1])
vocab = {}
for index, word in enumerate(vocab_list):
    vocab[word] = index
vocab_list

# LDA-SVI

## Funciones auxiliares

### Generación de atributos de cada texto (recuento de palabras)

In [ ]:
def parseDocument(doc, vocab):
    wordslist = list()
    countslist = list()
    doc = doc.lower()
    tokens = wordpunct_tokenize(doc)

    dictionary = dict()
    for word in tokens:
        if word in vocab:
            wordtk = vocab[word]
            if wordtk not in dictionary:
                dictionary[wordtk] = 1
            else:
                dictionary[wordtk] += 1

    wordslist.append(list(dictionary.keys()))
    countslist.append(list(dictionary.values()))
    return (wordslist[0], countslist[0])

### Cálculo de distribuciones de probabilidad

In [ ]:
def dirichlet_expectation(alpha):
    '''
    For a vector theta ~ Dir(alpha), computes E[log(theta)] given alpha.

    Taken from https://github.com/blei-lab/onlineldavb/blob/master/onlineldavb.py
    '''
    if (len(alpha.shape) == 1):
        return (psi(alpha) - psi(n.sum(alpha)))
    return (psi(alpha) - psi(n.sum(alpha, 1))[:, n.newaxis])

def beta_expectation(a, b, k):
    mysum = psi(a + b)
    Elog_a = psi(a) - mysum
    Elog_b = psi(b) - mysum
    Elog_beta = n.zeros(k)
    Elog_beta[0] = Elog_a[0]
    # print Elog_beta
    for i in range(1, k):
        Elog_beta[i] = Elog_a[i] + n.sum(Elog_b[0:i])
        # print Elog_beta
    # print Elog_beta
    return Elog_beta

def plottrace(x, Y, K, n, perp):
    for i in range(K):
        plt.plot(x, Y[i], label = "Topic %i" %(i+1))

    plt.xlabel("Number of Iterations")
    plt.ylabel("Probability of Each topic")
    plt.legend()
    plt.title("Trace plot for topic probabilities")
    plt.savefig("temp/plot_%i_%i_%f.png" %(K, n, perp))

## Implementación de la LDA mediante SVI

Obtenemos la implementación de LDA mediante SVI de las fuentes mencionadas en el guión:
* https://github.com/qlai/stochasticLDA
* https://github.com/blei-lab/onlineldavb

In [ ]:
n.random.seed(10000001)
meanchangethresh = 1e-3
MAXITER = 10000

class SVILDA():
    """
        Arguments:
        K: Number of topics
        vocab: A set of words to recognize. When analyzing documents, any word
           not in this set will be ignored.
        D: Total number of documents in the population. For a fixed corpus,
           this is the size of the corpus. In the truly online setting, this
           can be an estimate of the maximum number of documents that
           could ever be seen.
        alpha: Hyperparameter for prior on weight vectors theta
        eta: Hyperparameter for prior on topics beta
        tau: A (positive) learning parameter that downweights early iterations
        kappa: Learning rate: exponential decay rate---should be between
             (0.5, 1.0] to guarantee asymptotic convergence.
        Note that if you pass the same set of D documents in every time and
        set kappa=0 this class can also be used to do batch VB.
    """

    def __init__(self, vocab, K, D, alpha, eta, tau, kappa, docs, iterations, parsed = False):
        self._vocab = vocab
        self._V = len(vocab)
        self._K = K
        self._D = D
        self._alpha = alpha
        self._eta = eta
        self._tau = tau
        self._kappa = kappa
        self._lambda = 1* n.random.gamma(100., 1./100., (self._K, self._V))
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = n.exp(self._Elogbeta)
        self._docs = docs
        self.ct = 0
        self._iterations = iterations
        self._parsed = parsed
        self._trace_lambda = {}
        for i in range(self._K):
            self._trace_lambda[i] = [self.computeProbabilities()[i]]
        self._x = [0]

    def updateLocal(self, doc): #word_dn is an indicator variable with dimension V
        (words, counts) = doc
        newdoc = []
        N_d = sum(counts)
        phi_d = n.zeros((self._K, N_d))
        gamma_d = n.random.gamma(100., 1./100., (self._K))
        Elogtheta_d = dirichlet_expectation(gamma_d)
        expElogtheta_d = n.exp(Elogtheta_d)
        for i, item in enumerate(counts):
            for j in range(item):
                newdoc.append(words[i])
        assert len(newdoc) == N_d, "error"

        for i in range(self._iterations):
            for m, word in enumerate(newdoc):
                phi_d[:, m] = n.multiply(expElogtheta_d, self._expElogbeta[:, word]) + 1e-100
                phi_d[:, m] = phi_d[:, m]/n.sum(phi_d[:, m])

            gamma_new = self._alpha + n.sum(phi_d, axis = 1)
            meanchange = n.mean(abs(gamma_d - gamma_new))
            if (meanchange < meanchangethresh):
                break

            gamma_d = gamma_new
            Elogtheta_d = dirichlet_expectation(gamma_d)
            expElogtheta_d = n.exp(Elogtheta_d)

        newdoc = n.asarray(newdoc)
        return phi_d, newdoc, gamma_d

    def updateGlobal(self, phi_d, doc):

        lambda_d = n.zeros((self._K, self._V))

        for k in range(self._K):
            phi_dk = n.zeros(self._V)
            for m, word in enumerate(doc):
                phi_dk[word] += phi_d[k][m]
            lambda_d[k] = self._eta + self._D * phi_dk
        rho = (self.ct + self._tau) **(-self._kappa)
        self._lambda = (1-rho) * self._lambda + rho * lambda_d
        self._Elogbeta = dirichlet_expectation(self._lambda)
        self._expElogbeta = n.exp(self._Elogbeta)

        if self.ct % 10 == 9:
            for i in range(self._K):
                self._trace_lambda[i].append(self.computeProbabilities()[i])
            self._x.append(self.ct)

    def runSVI(self):
        for i in tqdm(range(self._iterations)):
            randint = random.randint(0, self._D-1)
            if self._parsed == False:
                doc = parseDocument(self._docs[randint],self._vocab)
            phi_doc, newdoc, gamma_d = self.updateLocal(doc)
            self.updateGlobal(phi_doc, newdoc)
            self.ct += 1

    def computeProbabilities(self):
        prob_topics = n.sum(self._lambda, axis = 1)
        prob_topics = prob_topics/n.sum(prob_topics)
        return prob_topics

    def getTopics(self, docs = None):
        prob_topics = self.computeProbabilities()
        prob_words = n.sum(self._lambda, axis = 0)

        if docs == None:
            docs = self._docs
        results = n.zeros((len(docs), self._K))
        for i, doc in enumerate(docs):
            parseddoc = parseDocument(doc, self._vocab)

            for j in range(self._K):
                aux = [self._lambda[j][word]/prob_words[word] for word in parseddoc[0]]
                doc_probability = [n.log(aux[k]) * parseddoc[1][k] for k in range(len(aux))]
                results[i][j] = sum(doc_probability) + n.log(prob_topics[j])
        finalresults = n.zeros(len(docs))
        for k in range(len(docs)):
            finalresults[k] = n.argmax(results[k])
        return finalresults, prob_topics

    def calcPerplexity(self, docs = None):
        perplexity = 0.
        doclen = 0.
        if docs == None:
            docs =  self._docs
        for doc in docs:
            parseddoc = parseDocument(doc, self._vocab)
            _, newdoc, gamma_d = self.updateLocal(parseddoc)
            approx_mixture = n.dot(gamma_d, self._lambda)
            # print(n.shape(approx_mixture))
            approx_mixture = approx_mixture / n.sum(approx_mixture)
            log_doc_prob = 0.
            for word in newdoc:
                log_doc_prob += n.log(approx_mixture[word])
            perplexity += log_doc_prob
            doclen += len(newdoc)
            # print(perplexity, doclen)
        perplexity = n.exp( - perplexity / doclen)
        print(perplexity)
        return perplexity

    def plotTopics(self, perp):
        plottrace(self._x, self._trace_lambda, self._K, self._iterations, perp)

def test(k, iterations):

    docs = getalldocs("alldocs2.txt")
    vocab = getVocab("dictionary2.csv")

    testset = SVILDA(vocab = vocab, K = k, D = len(docs), alpha = 0.2,
                     eta = 0.2, tau = 1024, kappa = 0.7, docs = docs,
                     iterations= iterations)
    testset.runSVI()
    finallambda = testset._lambda

    heldoutdocs = getalldocs("testdocs.txt")
    perplexity = testset.calcPerplexity(docs = heldoutdocs)

    with open("temp/%i_%i_%f_results.csv" %(k, iterations, perplexity), "w+") as f:
        writer = csv.writer(f)
        for i in range(k):
            bestwords = sorted(range(len(finallambda[i])), key=lambda j:finallambda[i, j])
            bestwords.reverse()
            writer.writerow([i])
            for j, word in enumerate(bestwords):
                writer.writerow([word, vocab.keys()[vocab.values().index(word)]])
                if j >= 15:
                    break
    topics, topic_probs = testset.getTopics()
    testset.plotTopics(perplexity)

    for kk in range(0, len(finallambda)):
        lambdak = list(finallambda[kk, :])
        lambdak = lambdak / sum(lambdak)
        temp = zip(lambdak, range(0, len(lambdak)))
        temp = sorted(temp, key = lambda x: x[0], reverse=True)
        # print temp
        print('topic %d:' % (kk))
        # feel free to change the "53" here to whatever fits your screen nicely.
        for i in range(0, 10):
            print('%20s  \t---\t  %.4f' % (vocab.keys()[vocab.values().index(temp[i][1])], temp[i][0]))

    with open("temp/%i_%i_%f_raw.txt" %(k, iterations, perplexity), "w+") as f:
        # f.write(finallambda)
        for result in topics:
            f.write(str(result) + " \n")
        f.write(str(topic_probs) + " \n")

# Resultados

Una vez definidas todas las funciones necesarias sólo queda ejecutar el análisis de LDA:

(este análisis le llevará unos 25 minutos... ¡tenga paciencia!)

In [ ]:
mode = 'normal'
K = 5
alpha = 0.2
eta = 0.2
tau = 1024
kappa = 0.7
iterations = 100000

if mode == "test":
    test(K, iterations)
if mode == "normal":
    docs = df.abstract.to_list()
    D = len(docs)
    print('number of docs: {}'.format(D))
    lda = SVILDA(vocab = vocab, K = K, D = D, alpha = alpha,
                 eta = eta, tau = tau, kappa = kappa, docs = docs,
                 iterations = iterations)
    lda.runSVI()
    lda

Podemos observar las distribución de probabilidades de cada palabra del vocabulario con respecto a cada tema identificado en los textos:

In [ ]:
lambda_df = pd.DataFrame({'word':list(vocab.keys())})
for i in range(K):
    lambda_df['Topic {}'.format(i+1)] = lda._lambda[i,:]
lambda_df

Para entender cómo el algoritmo de LDA ha calculado la distribución de probabilidades de cada tema podemos observar las 10 palabras con mayor probabilidad por tema:

In [ ]:
lambda_df.dtypes

In [ ]:
finalresults, prob_topics = lda.getTopics()

for column,prob in list(zip(lambda_df.columns[1:],prob_topics)):
    print('{} probability: {:.2f}%'.format(column,prob*100))
    print(lambda_df.nlargest(10, [column])[['word',column]],'\n')

Finalmente, también podemos observar cómo ha quedado la distribución de temas sobre el conjunto de los documentos descargados:

In [ ]:
labels = lambda_df.columns[1:]
sizes = prob_topics*100
explode = (0.1, 0.1, 0.1,0.1, 0.1)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

# Tarea adicional

Intentar reproducir el Gráfico 2 del guión del Caso de estudio 1.2, donde se analiza qué temas predominan en cada laboratorio.

In [ ]:
# Código